In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import scipy.stats as stats

In [ ]:
crimeHousing = pd.read_csv('crime-housing-austin-2015.csv')
zipCodes = pd.read_csv('AustinZipCodes.csv')
pd.set_option('display.max_columns', None)

# Crime per Month

# Housing vs Crime

In [ ]:
crimeAustin = crimeHousing[['Highest_Offense_Desc', 'Highest_NIBRS_UCR_Offense_Description', 'Clearance_Status', 'District', 'Zip_Code_Crime']].reset_index()
crimeAustin = crimeHousing[crimeHousing.Zip_Code_Housing.notna()]

In [ ]:
sns.countplot(x='Highest_NIBRS_UCR_Offense_Description', data=crimeAustin)
plt.xticks(rotation=60)

In [ ]:
theft = crimeAustin[(crimeAustin.Highest_NIBRS_UCR_Offense_Description == "Theft")]
theft = theft[theft.Zip_Code_Housing.notna()]
sns.displot(x='Zip_Code_Crime', data=theft)
plt.xticks(rotation=60)

In [ ]:
housing = crimeHousing.copy()
housing = crimeHousing[crimeHousing.Zip_Code_Housing.notna()]

In [ ]:
housing = crimeHousing[['Council_District', 'Zip_Code_Housing', 'Medianhouseholdincome']].reset_index()
housing = housing.sort_values(by='Medianhouseholdincome', ascending=True)
housing

In [ ]:
sns.scatterplot(data=housing, x='Zip_Code_Housing', y='Medianhouseholdincome' )